<a href="https://colab.research.google.com/github/brunaruppruela/mestrado_IFES_IA/blob/main/exercicioMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import gzip
import struct
import cv2
import urllib.request
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Função para baixar e carregar a base de dados MNIST diretamente do site do criador Yann LeCun
def load_mnist():
    def download_data(url, num_images):
        with urllib.request.urlopen(url) as response:
            with gzip.GzipFile(fileobj=response) as bytestream:
                bytestream.read(16)
                buf = bytestream.read(28 * 28 * num_images)
                data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
                data = data.reshape(num_images, 28, 28, 1)
                return data

    def download_labels(url, num_images):
        with urllib.request.urlopen(url) as response:
            with gzip.GzipFile(fileobj=response) as bytestream:
                bytestream.read(8)
                buf = bytestream.read(1 * num_images)
                labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
                return labels

    x_train = download_data('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 60000)
    y_train = download_labels('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 60000)
    x_test = download_data('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 10000)
    y_test = download_labels('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 10000)

    return (x_train, y_train), (x_test, y_test)

# Carregar a base de dados MNIST
(x_train, y_train), (x_test, y_test) = load_mnist()

# Normalizar os dados
x_train, x_test = x_train / 255.0, x_test / 255.0

# Dividir os dados de treinamento em treinamento e validação
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Definir a arquitetura da rede neural convolucional
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Avaliar o modelo nos dados de teste
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Acurácia nos dados de teste:', test_acc)

: 